# SIMULACION SALA DE ESPERA DE UN HOSPITAL

* Lo que hay que hacer.
Videoblog e informe en español y en ingles. Con su respectivo codigo fuente.

* Variables de interes.
    * Nombre del hospital: DISTRITO CIUDAD DE PAUTE
    * Numero de pacientes analizados: Es una variable aleatoria con funcion de distribucion de poisson.
    * Cuando llega el paciente al hospital: 10 - 60 minutos
    * Tiempos en la sala de espera: 12 - 45 minutos. Distribucion exponencial. Promedio (12 + 45) /2 = 22.5
    * Examenes: 2 horas. Minutos 120. Intervalo entre -60 a +60.
    * Paciente va a la sala de espera, de la sala de espera va hacer examenes.
    * Numeros de cabinas. 1 cabina. Esto es un recurso.

In [44]:
import simpy
import random

class Hospital:
    
    def __init__(self, env, tatencion, tot_sitios):
        self.env = env #Guardo el entorno
        self.tatencion = tatencion #Guardo mi tiempo de atencion
        self.tot_sitios = tot_sitios #Sitios que va a ser atendido la persona. Es un numero.
        
        self.sitios = simpy.Resource(self.env,self.tot_sitios) #Defino los sitios como fisicos.
    
    
    def atender_paciente(self, paciente):
        print('Llega paciente [',paciente,'] t =',self.env.now) #Capturo tiempo que fue atendido
        yield self.env.timeout(random.randint(self.tatencion-11, self.tatencion+11)) #Promedio atendido aleatorio
        print('Paciente atendido ','[',paciente,']')
        print('===============================================')
        
class Simulacion:
    
    def __init__(self,env, tatencion, tot_sitios):
        self.env = env
        self.hospital = Hospital(self.env, tatencion, tot_sitios) #Creo mi hospital
        
    def llega_paciente(self, paciente):
        #Crea bloques para pedir sitio cuando llega un paciente
        with self.hospital.sitios.request() as sitio:
            yield sitio #Ocupo el sitio, programacion concurrente.
            yield self.env.process(self.hospital.atender_paciente(paciente)) #Atiendo el paciente
            print('Atendio Pac. [',paciente,'] t= ',self.env.now)
            #Cuando termina el proceso, se libera solo
            
    def ejecutar_simulacion(self):
        for i in range(1000): # Llegan mil pacientes
            yield self.env.timeout(random.randint(10,60)) # Mejorar esto llega el paciente entre 10 a 60 minutos
            self.env.process(self.llega_paciente('Persona['+str(i+1)+']'))
                
        j = 0
        while True:
            j+=1
            self.env.process(self.hospital.llega_paciente('Persona['+str(i+1)+']'))
                
if __name__=="__main__":
    env = simpy.Environment()
    sim = Simulacion(env, 22, 1) #Tiempo de atencion = 22 minutos, Numero de cabinas = 1.
    env.process(sim.ejecutar_simulacion())
    env.run(until = 1000)

Llega paciente [ Persona[1] ] t = 11
Paciente atendido  [ Persona[1] ]
Atendio Pac. [ Persona[1] ] t=  28
Llega paciente [ Persona[2] ] t = 56
Paciente atendido  [ Persona[2] ]
Atendio Pac. [ Persona[2] ] t=  84
Llega paciente [ Persona[3] ] t = 102
Paciente atendido  [ Persona[3] ]
Atendio Pac. [ Persona[3] ] t=  129
Llega paciente [ Persona[4] ] t = 129
Paciente atendido  [ Persona[4] ]
Atendio Pac. [ Persona[4] ] t=  145
Llega paciente [ Persona[5] ] t = 179
Paciente atendido  [ Persona[5] ]
Atendio Pac. [ Persona[5] ] t=  209
Llega paciente [ Persona[6] ] t = 229
Paciente atendido  [ Persona[6] ]
Atendio Pac. [ Persona[6] ] t=  250
Llega paciente [ Persona[7] ] t = 265
Paciente atendido  [ Persona[7] ]
Atendio Pac. [ Persona[7] ] t=  284
Llega paciente [ Persona[8] ] t = 316
Paciente atendido  [ Persona[8] ]
Atendio Pac. [ Persona[8] ] t=  329
Llega paciente [ Persona[9] ] t = 335
Paciente atendido  [ Persona[9] ]
Atendio Pac. [ Persona[9] ] t=  350
Llega paciente [ Persona[10] ] t